<a href="https://colab.research.google.com/github/irene1125/Chatbot_SH/blob/main/Chatbot_practice1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터 읽어오기
train_file = "/content/drive/MyDrive/ChatbotData.csv"
data = pd.read_csv(train_file, delimiter=",")
features = data['Q'].tolist()
labels = data['label'].tolist()

In [4]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index

In [5]:
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen= MAX_SEQ_LEN, padding = 'post')


In [6]:
# 학습용, 검증용, 테스터용 데이터셋 생성
# 학습셋:검증셋:테이터셋 = 7:2:1

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

In [7]:
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs)*0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size+val_size).take(test_size).batch(20)

In [8]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1

In [10]:
#CNN 모델 정의
input_layer = Input(shape = (MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(
    filters = 128,
    kernel_size = 3,
    padding = 'valid',
    activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

conv2 = Conv1D(
    filters = 128,
    kernel_size = 4,
    padding = 'valid',
    activation = tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

conv3 = Conv1D(
    filters = 128,
    kernel_size = 5,
    padding = 'valid',
    activation = tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)


In [13]:
# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])
hidden = Dense(128, activation=tf.nn.relu)(concat)
droupout_hidden = Dropout(rate = dropout_prob)(hidden)
logits = Dense(3, name = 'logits')(droupout_hidden)
predictions = Dense(3, activation = tf.nn.softmax)(logits)

In [14]:
#모델 생성
model = Model(inputs = input_layer, outputs = predictions)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])
#모델 학습
model.fit(train_ds, validation_data = val_ds, epochs=EPOCH, verbose = 1)

#모델 평가
loss, accuracy = model.evaluate(test_ds, verbose = 1)
print('Accuracy: %f' %(accuracy * 100))
print('loss: %f' %(loss))

model.save('cnn_model.h5')

Epoch 1/5
414/414 [==============================] - 14s 32ms/step - loss: 0.9079 - accuracy: 0.5594 - val_loss: 0.5787 - val_accuracy: 0.8084
Epoch 2/5
414/414 [==============================] - 13s 31ms/step - loss: 0.5337 - accuracy: 0.7946 - val_loss: 0.2521 - val_accuracy: 0.9184
Epoch 3/5
414/414 [==============================] - 13s 31ms/step - loss: 0.3153 - accuracy: 0.8952 - val_loss: 0.1689 - val_accuracy: 0.9467
Epoch 4/5
414/414 [==============================] - 13s 32ms/step - loss: 0.1983 - accuracy: 0.9380 - val_loss: 0.0912 - val_accuracy: 0.9712
Epoch 5/5
60/60 [==============================] - 0s 4ms/step - loss: 0.0638 - accuracy: 0.9772
Accuracy: 97.715735
loss: 0.063793
